# 7. Tutorial 4 and 5: Descriptive analysis and data visualisation

#### Naoki TANI
#### Center for Advanced Policy Studies (CAPS), Institute of Economic Research, Kyoto University
#### April 25 and May 9, 2024

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## 1. Descriptive analysis with pandas

### 1-1. Pandas `Series` and `DataFrame`

#### [pandas](https://pandas.pydata.org/) is a python library providing a dataframe object to help you manage data. This lecture covers how to use `Series` and `DataFrame` objects to handle data.
#### `Series` is a one-dimensional array holdking any type of indexed data. All the data in a Series is of the same data type.

In [2]:
pref = ['Kochi', 'Ehime', 'Kagawa', 'Tokushima']
prefecture = pd.Series(pref)
prefecture # The values are labeled with their index number. First value has index 0, second value has index 1 etc.

0        Kochi
1        Ehime
2       Kagawa
3    Tokushima
dtype: object

#### The `Series` is composed of the values and index attributes.

In [3]:
print(prefecture.values)
print(prefecture.index)

['Kochi' 'Ehime' 'Kagawa' 'Tokushima']
RangeIndex(start=0, stop=4, step=1)


In [4]:
prefecture[1]

'Ehime'

#### `DataFrame` is a two-dimensional table of data with columns and rows. The columns are made up of `Series` objects.

In [5]:
prefstat = pd.DataFrame({'population':[736000,1396000,980000,763000],
                          'gdp':[2349510000000,4756495000000,3672273000000,3012328000000],
                          'income':[1866110000000,3516676000000,2835364000000,2219318000000]},
                           index=prefecture)
prefstat

,population,gdp,income
Kochi,736000,2349510000000,1866110000000
Ehime,1396000,4756495000000,3516676000000
Kagawa,980000,3672273000000,2835364000000
Tokushima,763000,3012328000000,2219318000000


In [6]:
prefstat.values
prefstat.index
prefstat.columns

Index(['population', 'gdp', 'income'], dtype='object')

In [7]:
prefstat['gdp_per_capita'] = prefstat['gdp'] / prefstat['population']
prefstat

,population,gdp,income,gdp_per_capita
Kochi,736000,2349510000000,1866110000000,3.192269e+06
Ehime,1396000,4756495000000,3516676000000,3.407231e+06
Kagawa,980000,3672273000000,2835364000000,3.747217e+06
Tokushima,763000,3012328000000,2219318000000,3.948005e+06


In [8]:
prefstat.population

Kochi         736000
Ehime        1396000
Kagawa        980000
Tokushima     763000
Name: population, dtype: int64

### 1-2. Extract relevant columns and rows

#### Now we use "令和３年経済センサス‐活動調査" to learn data manipulation using Pandas.
#### URL: https://www.e-stat.go.jp/stat-search/files?page=1&toukei=00200553&tstat=000001145590
#### "事業所に関する集計" > "産業横断的集計" > "事業所数、従業者数" > "6-1 産業(中分類)別民営事業所数、男女別従業者数、常用雇用者数、出向・派遣従業者数及び事業従事者数－全国、都道府県、市区町村"

In [ ]:
## We can read CSV files by using a pandan read_csv method.
establishment = pd.read_csv('R3census_b1_006_1.csv')
establishment

#### We can access elements of `DataFrame` using `iloc[]` and `loc` methods: we use `iloc[]` to access elements by integer index and `loc[]` to access by the index value.

In [ ]:
establishment.iloc[0,-10]
establishment.iloc[0,0]

#### We can slice columns and rows by using ":" operator as with Python lists and Numpy arrays.
#### ":" indicates `[start : end : step]`. The slice begins at the `start` index and takes steps of size `step` untill it reaches just before the end index.

In [ ]:
establishment.iloc[:,0]
#establishment.iloc[0:10:2,:2]

In [ ]:
establishment.loc[0,'Unnamed: 1']
establishment.loc[:,'Unnamed: 1']

#### We want to drop some unnecessary rows by specifying index values.

In [ ]:
establishment.drop(index=[0,1,2,3,4], inplace=True)

# We can also do it by specifying integer index.
#establishment.drop(index=establishment.index[[0,1,2,3,4]], inplace=True)

In [ ]:
# replace values of some cells in order to use the column of index "0" as a new column index. 
establishment.iloc[0,:5] = establishment.iloc[2,:5]
establishment.set_axis(establishment.iloc[0,:], axis=1, inplace=True)
establishment

establishment.drop(index=[5,6,7], inplace=True)
establishment

In [ ]:
# Extract relevant columns
establishment = establishment.loc[:,['地域区分','産業大分類','産業中分類','事業所数']]
establishment

#### We can split strings around given separator by using `str.split()`.

In [ ]:
#establishment['地域区分'].str.split('_', expand=True)
establishment = pd.concat([establishment, establishment['地域区分'].str.split('_', expand=True)], axis=1)
establishment.rename(columns={0: 'city_code', 1: 'city'}, inplace=True)
establishment.drop('地域区分',axis=1,inplace=True)
establishment

### 1-3. Convert data types

In [ ]:
# "dtypes" method returns the data type of each column.
establishment.dtypes

In [ ]:
# convert data type
establishment['city_code'] = establishment['city_code'].astype('int64')
establishment

In [ ]:
# error due to "-"
#establishment['事業所数'].str.replace(',', '').astype(int)

In [ ]:
# Convert object to integer
establishment['事業所数'] = establishment['事業所数'].str.replace(',', '')
establishment['事業所数'] = establishment['事業所数'].replace('-','0').astype('int64')
establishment

### 1-4. Boolean indexing
#### We can filter a `DataFrame` according to a set of criteria by indexing the data with boolean values.

In [ ]:
establishment.loc[establishment['city_code']==0]

#### We can use multiple boolean conditions by using `&` (and), `|` (or), and `~` (not).

In [ ]:
establishment.loc[(establishment['city_code']>=1) | (~establishment['city_code']==0)]

#### We want to get data of cities in `高知県`, `高松市`, `徳島市`, and `松山市`. 

In [ ]:
establishment = establishment.loc[((establishment['city_code']>=39201) & (establishment['city_code']<=39428)) | #高知県内の市町村
                                  (establishment['city_code']==37201) | #高松市
                                  (establishment['city_code']==36201) | #徳島市
                                  (establishment['city_code']==38201)   #松山市
                                 ]

In [ ]:
# Get number of establishments of "卸売業、小売業"
establishment = establishment.loc[establishment['産業大分類']=='I']

In [ ]:
# Rename the column names
establishment.rename(columns={'事業所数':'num_establishments_retail'}, inplace=True)

# Drop unnecessary columns
establishment.drop(['産業大分類','産業中分類','city_code'], axis=1, inplace=True)

In [ ]:
# We will merge this DataFrame with consumption data later.
establishment

## 2. Working with consumption data
### 2-1. Read the data

In [ ]:
# We can read CSV files by using a pandan read_csv method.
df = pd.read_csv('consumption_data.csv')

# drop rows if all values in the rows are NaN
df.dropna(how='all', inplace=True)

# rename column name
df.rename(columns={'決済日 の週':'week',
                 '業種①':'industry',
                 '性別':'sex',
                 '年代':'age',
                 '市区町村':'city',
                 '居住地方①':'residence',
                 '人数':'num_user',
                 '金額':'value',
                 '件数':'num_uses'}, inplace=True)
df;

In [ ]:
#import datatime module
from datetime import datetime

#### We can convert "年月日" string to datetime by using datetime module.

In [ ]:
df['week'] = df['week'].map(lambda x: datetime.strptime(x,'%Y年%m月%d日'))

In [ ]:
df

In [ ]:
# Add "year" column
df['year'] = 2023
df.loc[df['week']<pd.to_datetime('2020-1-1'), 'year'] = 2019
df

In [ ]:
# Convert object to integer
df['value'] = df['value'].str.replace(',', '').astype(int)

# "秘匿項目"
df['num_user'] = df['num_user'].replace(0.0,np.nan)
df['num_uses'] = df['num_uses'].replace(0.0,np.nan)

df.dtypes

In [ ]:
# Correct wrong city name
df['city'] = df['city'].replace('中村市','四万十市')
df['city'] = df['city'].replace('香美郡赤岡町','香南市')
df['city'] = df['city'].replace('香美郡土佐山田町','香美市')
df['city'] = df['city'].replace('吾川郡伊野町','吾川郡いの町')

In [ ]:
# Remove "~~郡" from city column
import re

df['city'] = df['city'].map(lambda x: re.sub('吾川郡','',x))
df['city'] = df['city'].map(lambda x: re.sub('安芸郡','',x))
df['city'] = df['city'].map(lambda x: re.sub('幡多郡','',x))
df['city'] = df['city'].map(lambda x: re.sub('高岡郡','',x))
df['city'] = df['city'].map(lambda x: re.sub('長岡郡','',x))

#np.unique(df.city)

### 2-2. Combine multiple DataFrames

#### We can merge multiple DataFrames by `merge()`, `.join()`, or`concat()`:
#### `merge()` combines data on common columns or indices.
#### `.join()` combines data on a key column or an index.
#### `concat()` combines DataFrames across rows or columns.
#### See https://pandas.pydata.org/docs/user_guide/merging.html .

In [ ]:
df = pd.merge(df, establishment)

In [ ]:
#df.loc[a.num_establishments_retail.isnull()]
df

### 2-3. Grouping and aggregation

#### We can use `.groupby()` method to specify the grouping variables and ways to aggregate data for each group.

In [ ]:
# '.groupby()' just creates a grouped dataframe without any printing method. 
df.groupby('city')

#### It is possible to specify aggregation methods.

In [ ]:
df.groupby('city').value.sum();

df_gr = df.groupby(['city','year','num_establishments_retail']).agg({'value': 'sum', 'num_user':'mean'})
df_gr

# Transform multi-index to single index
df_gr.reset_index(level=['city','num_establishments_retail'], inplace=True)
df_gr

### 2-4. Scatterplot

In [ ]:
figure, ax  = plt.subplots(dpi=120)
ax.scatter(df_gr['num_establishments_retail'], df_gr['value']/1000000)
###########################################################################
ax.legend(bbox_to_anchor=(1.05, 0.5), prop = {'family' : 'MS Gothic'}, loc='center left') #place legend outside the plot
#ax.set_xlim(0, 1000)
#ax.set_ylim(0, 1000)
ax.set_ylabel("消費額（百万円）", fontname = 'MS Gothic')
ax.set_xlabel("事業所数", fontname = 'MS Gothic')
#ax.set_xlabel("percentile of industrial group earnings distribution")
ax.set_title("卸売業・小売業の事業所数と消費額の関係", pad = 20, fontname = 'MS Gothic')
figure.subplots_adjust(top=1.05, bottom=0.15) 

In [ ]:
df_gr

## 3. Simple linear regression
### 3-1. Derivation of the coefficient

#### Now we derive the equations of simple linear regression (SLR) model. SLR is mainly used to describe the linear dependence of one variable on another.
#### Linear regression estimates the best-fit line through a scatterplot of the data by minimizing the difference between the actual data and predicted value of the line.

#### Consider a set of $n$ points $(X_i, Y_i)$, we want to find the best-fit line $\hat{Y}_i = \beta_0 + \beta_1 X_i$ to minimize the sum of squared errors $\sum^{n}_{i=1}(Y_i - \hat{Y}_i)^2$ 

#### Define the sum of squared errors as 
#### $$L(\beta_0, \beta_1) = \sum^{n}_{i=1}(Y_i - \hat{Y}_i)^2 = \sum^{n}_{i=1}(Y_i - \beta_0 - \beta_1 X_i)^2$$
#### Since we want to minimize $L(\beta_0, \beta_1)$, we set the derivatives of $L$ with respect to $\beta_0$ and $\beta_1$ to $0$.
#### $$\frac{\partial L}{\partial \beta_0}= 2(n\beta_0 + \beta_1 \sum^n_{i=1}X_{i} - \sum^n_{i=1}Y_{i}) = 0$$
#### Then, we derive 
#### $$\beta_0 = \bar{Y} - \beta_1 \bar{X}$$

#### Next, we use the derivatives of $L$ with respect to $\beta_1$.
#### $$\frac{\partial L}{\partial \beta_1}= \sum^n_{i=1} -2( X_i Y_i -\beta_0 X_i - \beta_1 X_{i}^2) = 0$$
#### Substituting $\beta_0$ we derived above to the equation $\frac{\partial L}{\partial \beta_1}=0$ yields
### $$\beta_1 = \frac{\sum^{n}_{i=1}(X_i Y_i - X_i \bar{Y})}{\sum^{n}_{i=1}(X_{i}^2)-X_i \bar{X}}$$

In [ ]:
y = df_gr['value']
x = df_gr['num_establishments_retail']

In [ ]:
sum(x.iloc[i]*y.iloc[i]-np.mean(y)*x.iloc[i] for i, v in enumerate(x)) / sum(x.iloc[i]**2-np.mean(x)*x.iloc[i] for i, v in enumerate(x))

In [ ]:
from statsmodels.formula.api import ols, logit, probit
from statsmodels.iolib.summary2 import summary_col

In [ ]:

formula = 'value ~ 1 + num_establishments_retail'
model = ols(formula, data=df_gr).fit()


In [ ]:
res = summary_col(model, stars=True, float_format='%0.4f',
                           info_dict={'N': lambda x: "{0:d}".format(int(x.nobs))},drop_omitted=True)
print(res)